#Dataset 1

In [ ]:
import numpy as np
import pandas as pd
from sklearn import metrics
import torch
from torch.utils.data import Dataset, DataLoader, RandomSampler, SequentialSampler

In [ ]:
!pip install -q convokit

     |████████████████████████████████| 133kB 6.5MB/s 
     |████████████████████████████████| 1.4MB 8.5MB/s 
     |████████████████████████████████| 245kB 32.6MB/s 
     |████████████████████████████████| 71kB 6.6MB/s 


In [ ]:
from convokit import Corpus, download
corpus = Corpus(filename=download("friends-corpus"))

corpus.print_summary_stats()
data2 = corpus.get_utterances_dataframe()


data1 = data2[['text', 'speaker']]
data1 = data1.reset_index()
data1 = data1.drop('id',axis=1)


np.savetxt(r'test.txt', data1.values, fmt='%s')
print(data1)

Number of Speakers: 700
Number of Utterances: 67373
Number of Conversations: 3107
                                                    text          speaker
0      There's nothing to tell! He's just some guy I ...    Monica Geller
1      C'mon, you're going out with the guy! There's ...   Joey Tribbiani
2      All right Joey, be nice. So does he have a hum...    Chandler Bing
3                               Wait, does he eat chalk?    Phoebe Buffay
4                                                         TRANSCRIPT_NOTE
...                                                  ...              ...
67368                            Oh, it's gonna be okay.    Chandler Bing
67369  Do you guys have to go to the new house right ...     Rachel Green
67370                                  We got some time.    Monica Geller
67371                   Okay, should we get some coffee?     Rachel Green
67372                                       Sure. Where?    Chandler Bing

[67373 rows x 2 columns]


In [ ]:
for i in range(len(data1)):
    if(data1.iloc[i][-1] == 'Chandler Bing'):
        data1.loc[i][-1] = 'c'
    elif(data1.iloc[i][-1] == 'Rachel Green'):
        data1.loc[i][-1] = 'r'
    elif(data1.iloc[i][-1] == 'Ross Geller'):
        data1.loc[i][-1] = 'R'
    elif(data1.iloc[i][-1] == 'Monica Geller'):
        data1.loc[i][-1] = 'm'
    elif(data1.iloc[i][-1] == 'Phoebe Buffay'):
        data1.loc[i][-1] = 'p'
    elif(data1.iloc[i][-1] == 'Joey Tribbiani'):
        data1.loc[i][-1] = 'j'
    else:
        data1.loc[i][-1] = 'o'

data1.sample(frac=1) 
data1.head()



,text,speaker
0,There's nothing to tell! He's just some guy I ...,m
1,"C'mon, you're going out with the guy! There's ...",j
2,"All right Joey, be nice. So does he have a hum...",c
3,"Wait, does he eat chalk?",p
4,,o


In [ ]:

data1 = data1[data1['speaker'] != 'o']
print(data1.count())
print(data1)


text       51312
speaker    51312
dtype: int64
                                                    text speaker
0      There's nothing to tell! He's just some guy I ...       m
1      C'mon, you're going out with the guy! There's ...       j
2      All right Joey, be nice. So does he have a hum...       c
3                               Wait, does he eat chalk?       p
5      Just, 'cause, I don't want her to go through w...       p
...                                                  ...     ...
67368                            Oh, it's gonna be okay.       c
67369  Do you guys have to go to the new house right ...       r
67370                                  We got some time.       m
67371                   Okay, should we get some coffee?       r
67372                                       Sure. Where?       c

[51312 rows x 2 columns]


In [ ]:
data1 = data1.reset_index(drop=True)
print(data1)

                                                    text speaker
0      There's nothing to tell! He's just some guy I ...       m
1      C'mon, you're going out with the guy! There's ...       j
2      All right Joey, be nice. So does he have a hum...       c
3                               Wait, does he eat chalk?       p
4      Just, 'cause, I don't want her to go through w...       p
...                                                  ...     ...
51307                            Oh, it's gonna be okay.       c
51308  Do you guys have to go to the new house right ...       r
51309                                  We got some time.       m
51310                   Okay, should we get some coffee?       r
51311                                       Sure. Where?       c

[51312 rows x 2 columns]


#LSTM - nivel de caracter


In [ ]:
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt

import torch
import string
import random
import re
import time, math

import torch.utils.data as utils
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils
import seaborn as sn
from sklearn.metrics import confusion_matrix

import torchvision
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import collections

import torch.nn as nn
from torch.autograd import Variable
import torch
import torch.utils.data as utils
import csv

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [ ]:
datat = data1[50001:]
print(datat)

                                                    text speaker
50001          Yeah, you miss alot, when you're moo-ing.       m
50002  Oh! It was our pleasure. We are so much enjoyi...       m
50003  Oh, uhm, okay, uhm, do you mind if we ask you ...       c
50004                                      That's great.       c
50005                                    How's that now?       m
...                                                  ...     ...
51307                            Oh, it's gonna be okay.       c
51308  Do you guys have to go to the new house right ...       r
51309                                  We got some time.       m
51310                   Okay, should we get some coffee?       r
51311                                       Sure. Where?       c

[1311 rows x 2 columns]


In [ ]:
data1 = data1[:5000]
print(data1)

                                                   text speaker
0     There's nothing to tell! He's just some guy I ...       m
1     C'mon, you're going out with the guy! There's ...       j
2     All right Joey, be nice. So does he have a hum...       c
3                              Wait, does he eat chalk?       p
4     Just, 'cause, I don't want her to go through w...       p
...                                                 ...     ...
4995                  You gotta tell Ross how you feel.       j
4996  Come on. How can I just tell him? What about J...       r
4997  What about her? They've only been going out fo...       j
4998                        I don't know, I don't know.       r
4999  Look, Rach, Rach! I've been with my share of w...       j

[5000 rows x 2 columns]


In [ ]:
chunk_len = 150

# Gera um pedaço aleatório de texto com o tamanho especificado em chuck_len
def random_chunk():
    index = random.randint(0, len(data1)-1)
    
    data = data1.iloc[index][0]
    
    for i in range(200-len(data)):#normaliza os chunks com tamanho igual
        data+='/0'
        
    return data, index
    
print(random_chunk())

("Well, I'm off to Carol's./0/0/0/0/0/0/0/0/0/0/0/0/0/0/0/0/0/0/0/0/0/0/0/0/0/0/0/0/0/0/0/0/0/0/0/0/0/0/0/0/0/0/0/0/0/0/0/0/0/0/0/0/0/0/0/0/0/0/0/0/0/0/0/0/0/0/0/0/0/0/0/0/0/0/0/0/0/0/0/0/0/0/0/0/0/0/0/0/0/0/0/0/0/0/0/0/0/0/0/0/0/0/0/0/0/0/0/0/0/0/0/0/0/0/0/0/0/0/0/0/0/0/0/0/0/0/0/0/0/0/0/0/0/0/0/0/0/0/0/0/0/0/0/0/0/0/0/0/0/0/0/0/0/0/0/0/0/0/0/0/0/0/0/0/0/0/0/0/0/0/0/0/0/0/0", 1660)


In [ ]:
file = open('test.txt').read()
all_characters = ''.join(set(file))
n_characters = len(all_characters)
print(all_characters)
print(n_characters)
file_len = len(file)
print('file_len =', file_len)




a;R'-8cWJCiA_H"7 v6}
ny&4G#gNMhBSo5?{Z1$TD!fEUKk9Ye:+jbXmlwOqQxV`LruF[p%3(/>Pz20I]td.s)*,
89
file_len = 4169043


In [ ]:
def char_tensor(string):
    tensor = torch.zeros(len(string)).long()
    for c in range(len(string)):
        try:
            tensor[c] = all_characters.index(string[c])
        except:
            print(c)
            raise
    return Variable(tensor)

print(char_tensor('abcDEF'))

tensor([ 0, 54,  6, 41, 44, 68])


In [ ]:
# Gera um 'exemplo' aleatório (um pedaço aleatório convertido para lista de inteiros)
def random_training_set():    
    chunk, index = random_chunk()
    inp = char_tensor(chunk[:-1])
    target = data1.iloc[index][-1]
    target = char_tensor(target)
    return inp, target

print(random_training_set())
#tensor = random_training_set()
#print(tensor)
#if(tensor[0][199] == 3) :
   # print("sim!")

(tensor([ 7, 30,  0, 82, 35, 16,  7, 30,  0, 82, 35, 16,  7, 30,  0, 82, 35, 42,
        74, 79, 74, 79, 74, 79, 74, 79, 74, 79, 74, 79, 74, 79, 74, 79, 74, 79,
        74, 79, 74, 79, 74, 79, 74, 79, 74, 79, 74, 79, 74, 79, 74, 79, 74, 79,
        74, 79, 74, 79, 74, 79, 74, 79, 74, 79, 74, 79, 74, 79, 74, 79, 74, 79,
        74, 79, 74, 79, 74, 79, 74, 79, 74, 79, 74, 79, 74, 79, 74, 79, 74, 79,
        74, 79, 74, 79, 74, 79, 74, 79, 74, 79, 74, 79, 74, 79, 74, 79, 74, 79,
        74, 79, 74, 79, 74, 79, 74, 79, 74, 79, 74, 79, 74, 79, 74, 79, 74, 79,
        74, 79, 74, 79, 74, 79, 74, 79, 74, 79, 74, 79, 74, 79, 74, 79, 74, 79,
        74, 79, 74, 79, 74, 79, 74, 79, 74, 79, 74, 79, 74, 79, 74, 79, 74, 79,
        74, 79, 74, 79, 74, 79, 74, 79, 74, 79, 74, 79, 74, 79, 74, 79, 74, 79,
        74, 79, 74, 79, 74, 79, 74, 79, 74, 79, 74, 79, 74, 79, 74, 79, 74, 79,
        74, 79, 74, 79, 74, 79, 74, 79, 74, 79, 74, 79, 74, 79, 74, 79, 74, 79,
        74, 79, 74, 79, 74, 79, 74, 79,

In [ ]:
class RNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, n_layers=1):
        super(RNN, self).__init__()
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.n_layers = n_layers
        
        self.encoder = nn.Embedding(input_size, hidden_size)
        self.lstm = nn.LSTM(hidden_size, hidden_size, n_layers)
        self.decoder = nn.Linear(hidden_size, output_size)
    
    def forward(self, input, hidden, hidden2):
        input = self.encoder(input.view(1, -1))
        output, (hidden, hidden2) = self.lstm(input.view(1, 1, -1), (hidden, hidden2))
        decoded_output = self.decoder(output.view(1, -1))
        return decoded_output, hidden, hidden2

    def init_hidden(self):
        return Variable(torch.zeros(self.n_layers, 1, self.hidden_size))

In [ ]:
# Gera um texto iniciando com "prime_str" de tamanho "predict_len" e variação definida por "temperature"
def evaluate(prime_str='A', predict_len=1, temperature=0.8):
    hidden = rnn.init_hidden().cuda()
    hidden2 = rnn.init_hidden().cuda()
    prime_input = char_tensor(prime_str).cuda()
    predicted = ''

    # prime_str é o texto inicial que o gerado irá completar
    for p in range(len(prime_str) - 1):
        _, hidden, hidden2 = rnn(prime_input[p], hidden, hidden2)
    inp = prime_input[-1]
    
    for p in range(predict_len):
        output, hidden, hidden2 = rnn(inp, hidden, hidden2)
        
        # Usa a temperatura para amostrar a distribuição e escolher a saída probabilísticamente
        output_dist = output.data.view(-1).div(temperature).exp()
        top_i = torch.multinomial(output_dist, 1)[0]
        
        # Adiciona o caracter predito à string de saída
        predicted_char = all_characters[top_i]
        predicted += predicted_char
        inp = char_tensor(predicted_char).cuda()

    return predicted

In [ ]:
# Treina sobre um exemplo (i.e. uma amostragem do texto)

def train(inp, target):
    hidden = rnn.init_hidden()
    hidden2 = rnn.init_hidden()
    rnn.zero_grad()
    loss = 0

    inp = inp.cuda()
    hidden = hidden.cuda()
    hidden2 = hidden2.cuda()
    for c in range(chunk_len):
        if(inp[c] != 79 or inp[c] != 74):
            output, hidden, hidden2 = rnn(inp[c], hidden, hidden2)
            loss += loss_metric(output, target[0].unsqueeze(0))
        else:
            break

    loss.backward()
    optimizer.step()

    return loss.data.item() / chunk_len

In [ ]:

def time_since(since):
    s = time.time() - since
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)

In [ ]:
n_epochs = 10000
print_every = 100
hidden_size = 512
n_layers = 4
lr = 0.0001

cuda = torch.device('cuda')

rnn = RNN(n_characters, hidden_size, n_characters, n_layers)
rnn.cuda()

optimizer = torch.optim.Adam(rnn.parameters(), lr=lr)
loss_metric = nn.CrossEntropyLoss()

start = time.time()
all_losses = []
loss_avg = 0

for epoch in range(1, n_epochs + 1):
    example = random_training_set()
    #print(example[0])
    loss = train(example[0].cuda(),example[1].cuda())       
    loss_avg += loss

    if epoch % print_every == 0:
        print('[%s (%d %d%%) %.4f]' % (time_since(start), epoch, epoch / n_epochs * 100, loss))
       # print(evaluate('The', 100), '\n')

 53s (100 1%) 1.5868]
 47s (200 2%) 2.3636]
 41s (300 3%) 1.8955]
 34s (400 4%) 1.8995]
 28s (500 5%) 1.5700]
 21s (600 6%) 1.8301]
 15s (700 7%) 1.8844]
 9s (800 8%) 1.7960]
 2s (900 9%) 1.7110]
 55s (1000 10%) 2.0456]
 48s (1100 11%) 1.6487]
 42s (1200 12%) 1.8732]
 35s (1300 13%) 2.0228]
 28s (1400 14%) 2.1389]
 22s (1500 15%) 1.8270]
 15s (1600 16%) 1.6750]
 8s (1700 17%) 1.6697]
 1s (1800 18%) 2.0133]
 53s (1900 19%) 1.7400]
 46s (2000 20%) 1.7579]
 40s (2100 21%) 2.1276]
 33s (2200 22%) 1.9422]
 27s (2300 23%) 1.3426]
 20s (2400 24%) 1.8569]
 14s (2500 25%) 1.5104]
 7s (2600 26%) 1.7614]
 1s (2700 27%) 2.0682]
 54s (2800 28%) 1.8523]
 48s (2900 28%) 2.0649]
 41s (3000 30%) 1.5862]
 35s (3100 31%) 1.7370]
 28s (3200 32%) 1.6329]
 22s (3300 33%) 1.3977]
 15s (3400 34%) 2.5024]
 8s (3500 35%) 1.9550]
 1s (3600 36%) 1.6956]
 54s (3700 37%) 1.9600]
 47s (3800 38%) 1.6346]
 41s (3900 39%) 2.2375]
 34s (4000 40%) 1.7940]
 27s (4100 41%) 1.5568]
 21s (4200 42%) 1.6978]
 14s (4300 43%) 2.

##Teste LSTM caracter

In [ ]:
# carregar o dataset de test

In [ ]:
#usando o evaluate para avaliar o modelo com o dataset de test
datat = datat[datat['text'].map(len) > 15]

correct = 0
loss = 0

for i in range(len(datat)):
    predict = evaluate(datat.iloc[i][0])
    label = datat.iloc[i][-1]
    if(predict == label):
        correct+=1
    
        
        
        
accuracy = correct/(len(datat))



print('Acurácia: %d%%'  % (accuracy * 100))

Acurácia: 16%
